In [1]:
import pandas as pd, numpy as np
import itertools, requests, json

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
lm=pd.read_excel('data/szekelyhon_locs_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]
lm["uid"] = lm["title"].map(str) + lm["content"]

In [5]:
dc=dc.set_index('uid')[['relevant','severity','deaths','duplicate']]
dm=dm.set_index('uid')
lm=lm.set_index('uid')[['predicted_locs','actual_locs']]

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.join(dc).drop_duplicates()

In [8]:
dl=df.join(lm)

In [9]:
len(dl)

12

Working df ready. Now infer location.

In [10]:
to_geo=[i.split(',') for i in dl['actual_locs'].unique() if str(i)!='nan']

In [11]:
to_geo=[i.strip() for i in list(itertools.chain.from_iterable(to_geo))]

Load HU names

In [12]:
coords=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [13]:
data=[]
header=['index']
for i in dl.iteritems():
    header.append(i[0])
header.append('geo_loc')
header.append('latitude')
header.append('longitude')
header.append('id')
for i in dl.T.iteritems():
    dummy=[i[0]]
    for j in range(len(i[1])):
        if j==(len(i[1])-1):
            dummy.append(i[1][j])
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dtime=str(dummy[1])[:10]
            if dtime=='nan': dtime=u'N/A' 
            if str(i[1][j])!='nan':    
                for k in i[1][j].split(','):
                    tk=k.strip().replace('[','').replace(']','').replace("'",'')
                    dummy[-4]=tk
                    if tk in coords:
                        dummy[-3]=coords[tk][0]
                        dummy[-2]=coords[tk][1]
                    else:
                        dummy[-3]=''
                        dummy[-2]=''
                    #dummy[-1]=str(dummy[-3])+' ('+str(dummy[0]+1)+')\n'+dtime
                    dummy[-1]=str(dummy[-4])+', '+dtime
                    data.append(list(dummy))                    
            else:
                dummy[-1]=u'N/A' +', '+dtime
                data.append(list(dummy))
        else: 
            dummy.append(i[1][j])

In [14]:
df=pd.DataFrame(data)
df.columns=header
df

,index,Unnamed: 0,date,hudate,link,image,category,icategory,tags,title,...,relevant,severity,deaths,duplicate,predicted_locs,actual_locs,geo_loc,latitude,longitude,id
0,"Elpusztult egy medve Etéd határában, orvvadász...",0,2020-04-27,2020. április 27.,https://szekelyhon.ro/aktualis/elpusztult-egy-...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,['Etéd'],"Elpusztult egy medve Etéd határában, orvvadász...",...,1,2,0,0,Etéd,Etéd,Etéd,46.4484,25.0532,"Etéd, 0"
1,Célkeresztben a „medvetörvény” – a vadászok sz...,1,2020-05-04,2020. május 04.,https://szekelyhon.ro/aktualis/celkeresztben-a...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Hargita megye', 'Háromszék', 'Marosszék']",Célkeresztben a „medvetörvény” – a vadászok sz...,...,1,0,0,0,Haraly,NaN,,,,"N/A, 1"
2,"Tehetetlenek a lakók, az ablakból kénytelenek ...",2,2020-05-05,2020. május 05.,https://szekelyhon.ro/aktualis/tehetetlenek-a-...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Háromszék', 'medve']","Tehetetlenek a lakók, az ablakból kénytelenek ...",...,1,3,0,0,"['Ádámos', 'Zágon']",Zágon,Zágon,45.7717,26.1243,"Zágon, 2"
3,Három juhot ölt meg a medve egy farcádi gazda ...,3,2020-05-05,2020. május 05.,https://szekelyhon.ro/aktualis/harom-juhot-olt...,https://media.szekelyhon.ro/pictures/vasarhely...,aktualis,vasarhely,"['Felsőboldogfalva', 'medve']",Három juhot ölt meg a medve egy farcádi gazda ...,...,1,3,3,0,"['Felsőboldogfalva', 'Farcád']",Farcád,Farcád,46.3071,25.2361,"Farcád, 3"
4,Áthelyezik egy másik területre a Tusnádfürdőre...,4,2020-05-08,2020. május 08.,https://szekelyhon.ro/aktualis/athelyezik-egy-...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['medve', 'Tusnádfürdő']",Áthelyezik egy másik területre a Tusnádfürdőre...,...,1,0,0,0,Tusnádfürdő,Tusnádfürdő,Tusnádfürdő,46.1461,25.857,"Tusnádfürdő, 4"
5,Medvék miatt riasztották a hatóságokat és ébre...,5,2020-05-11,2020. május 11.,https://szekelyhon.ro/aktualis/medvek-miatt-ri...,https://media.szekelyhon.ro/pictures/123rf/202...,aktualis,123rf,"['Gyergyószék', 'medve', 'Udvarhelyszék']",Medvék miatt riasztották a hatóságokat és ébre...,...,1,2,0,0,Vasláb,Vasláb,Vasláb,46.6461,25.6237,"Vasláb, 5"
6,Külön megelőzési kvótát és a medvekárok kifize...,6,2020-05-12,2020. május 12.,https://szekelyhon.ro/aktualis/kulon-megelozes...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Kovászna megye', 'medve']",Külön megelőzési kvótát és a medvekárok kifize...,...,1,0,0,0,NaN,NaN,,,,"N/A, 6"
7,A kormorán lekerülhet a védett fajok listájáró...,7,2020-05-13,2020. május 13.,https://szekelyhon.ro/vilag/a-kormoran-lekerul...,https://media.szekelyhon.ro/pictures/csik/aktu...,vilag,csik,"['Belföld', 'medve']",A kormorán lekerülhet a védett fajok listájáró...,...,1,0,0,0,NaN,NaN,,,,"N/A, 7"
8,Medve hatolt be egy tusnádfürdői portáraEgy tu...,8,2020-05-15,2020. május 15.,https://szekelyhon.ro/aktualis/medve-hatolt-be...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['medve', 'Tusnád']",Medve hatolt be egy tusnádfürdői portára,...,1,1,0,0,"['Tusnád', 'Tusnádfürdő']",Tusnádfürdő,Tusnádfürdő,46.1461,25.857,"Tusnádfürdő, 8"
9,Hárombocsos anyamedve bukkant fel Gyergyószent...,9,2020-05-15,2020. május 15.,https://szekelyhon.ro/aktualis/harombocsos-any...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Gyergyószentmiklós', 'medve']",Hárombocsos anyamedve bukkant fel Gyergyószent...,...,1,1,0,0,Gyergyószentmiklós,Gyergyószentmiklós,Gyergyószentmiklós,46.7212,25.5855,"Gyergyószentmiklós, 9"


Save data for loading in viz app

In [15]:
df['checked']=''
df['exact_loc']=0
df['source']='Székelyhon'

In [16]:
df=df[['checked',
    'date',
    'link',
    'image',
    'source',
    'title',
    'content',
    'relevant',
    'severity',
    'deaths',
    'duplicate',
    'geo_loc',
    'exact_loc',
    'latitude',
    'longitude']].set_index('checked')

In [17]:
#PowerBI exploration data
df.to_excel('data/szekelyhon_data.xlsx')